In [ ]:
from datetime import datetime, timedelta
from pyspark.sql import functions as F

# Read in current device_events into a Dataframe
df = spark.read.parquet("/mnt/edl/raw/nmis_dc_logs/device_events.parquet")

# Append the current results into the archived device_events parquet
# TODO clean out this archive over a longer duration.
df.write.mode("append").parquet(
    "/mnt/edl/raw/nmis_dc_logs/archive/device_events.parquet"
)

# Get timestamps for the last 15 days
end_date = datetime.now()
start_date = end_date - timedelta(days=15)

# Filter the current results to include only the events from last 15 days
df = df.filter(
    (F.col("event_timestamp") >= start_date) & (F.col("event_timestamp") <= end_date)
)

# Overwrite the current device_events with the results from last 15 days.
df.write.mode("overwrite").parquet("/mnt/edl/raw/nmis_dc_logs/device_events.parquet")